<a href="https://colab.research.google.com/github/HIsu1231/AI_INOVATION_SQUARE/blob/master/CNN_5X5X32_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.4.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.0


In [2]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.8MB 47.5MB/s 
     |████████████████████████████████| 512kB 51.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=83a18e5b50f2c49e167414460366bdef193c700bdaf57721c0f8e8cbe87ce475
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

print("train images shape = ", mnist.train.images.shape)
print("train labels shape = ", mnist.train.labels.shape )
print("test images shape = ", mnist.test.images.shape)
print("test lables shape = ", mnist.test.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
train images shape =  (55000, 784)
train labels shape =  (55000, 10)
test images shape =  (10000, 784)
test lables shape =  (10000, 10)


In [37]:
i_nodes = mnist.train.images.shape[1]
o_nodes = mnist.train.labels.shape[1]

#hyper_parameter
learning_rate = 0.001
epochs = 30
batch_size = 100

X = tf.placeholder(tf.float32, [None,i_nodes])
T = tf.placeholder(tf.float32, [None,o_nodes])

A1 = tf.reshape(X, [-1,28,28,1]) #28X28X1

In [43]:
#5X5X32 Filter
W2 = tf.Variable(tf.random_normal([5,5,1,32], stddev = 0.01))
b2 = tf.Variable(tf.random_normal([32]))

#28X28X32
C2 = tf.nn.conv2d(A1, W2, strides =[1,1,1,1], padding='SAME')

Z2 = tf.nn.relu(C2+b2)

#14X14X32 (합성곱층에 zero-padding을 사용하며, 이 경우에는 출력 특성맵의 크기는 입력을 스트라이드로 나눈 다음 올림 한 것과 같다
A2 = tf.nn.max_pool(Z2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


A2_flat = tf.reshape(A2, [-1, 14*14*32])

In [52]:
W3 = tf.Variable(tf.random_normal([14*14*32,10],stddev=0.01))
b3 = tf.Variable(tf.random_normal([10]))

Z3 = tf.matmul(A2_flat,W3) + b3

A3 = tf.nn.softmax(Z3)

In [56]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=T))

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [54]:
predicted_val = tf.argmax(A3,1)

predicted = tf.cast(tf.equal(tf.argmax(A3,1), tf.argmax(T,1)), dtype=tf.float32)

accuracy = tf.reduce_mean(predicted)

In [59]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    start_time = datetime.now()

    false_list = []

    for i in range(epochs):

        total_batch = int(mnist.train.num_examples / batch_size)

        for step in range(total_batch):

            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)

            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})  
            
            if step % 100 == 0:

                print("epochs = ", i, ", step = ", step, ", loss value = ", loss_val)

    end_time = datetime.now()

epochs =  0 , step =  0 , loss value =  2.803931
epochs =  0 , step =  100 , loss value =  0.7037609
epochs =  0 , step =  200 , loss value =  0.40183643
epochs =  0 , step =  300 , loss value =  0.43934423
epochs =  0 , step =  400 , loss value =  0.30442315
epochs =  0 , step =  500 , loss value =  0.19881958
epochs =  1 , step =  0 , loss value =  0.25956953
epochs =  1 , step =  100 , loss value =  0.17470054
epochs =  1 , step =  200 , loss value =  0.268293
epochs =  1 , step =  300 , loss value =  0.19082674
epochs =  1 , step =  400 , loss value =  0.15264723
epochs =  1 , step =  500 , loss value =  0.096018486
epochs =  2 , step =  0 , loss value =  0.08531258
epochs =  2 , step =  100 , loss value =  0.08243517
epochs =  2 , step =  200 , loss value =  0.041273355
epochs =  2 , step =  300 , loss value =  0.027580056
epochs =  2 , step =  400 , loss value =  0.06339269
epochs =  2 , step =  500 , loss value =  0.05326423
epochs =  3 , step =  0 , loss value =  0.04812696
epo

KeyboardInterrupt: ignored

In [ ]:
    test_x_data = mnist.test.images
    test_t_data = mnist.test.labels

    for i in range(len(test_x_data)):

        predicted_val = sess.run([predicted_val], feed_dict={X:test_x_data[i], T: test_t_data[i]})

        print(predicted_val)

        predicted = sees.run([predicted], feed_dict={X: test_x_data[i], T: test_t_data[i]})

        if predicted == 0:

            false_list.append(i)
